In [ ]:
! pip install pytorch-tabnet

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import log_loss, roc_auc_score

import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.metrics import Metric


import matplotlib.pyplot as plt
import seaborn as sns
import os


from itertools import combinations
import random

from PIL import Image

import catboost as catt


import warnings
warnings.filterwarnings("ignore")
VERSION = 2

In [ ]:
base_path = "/kaggle/input/final-deepmind-comp-dataset/final_deepmind_comp_dataset/zindi_data/"
additional_path = "/kaggle/input/final-deepmind-comp-dataset/final_deepmind_comp_dataset/image_classifier_results/"
train = pd.read_csv(base_path + "Train.csv")
test = pd.read_csv(base_path + "Test.csv")
train_with_cv_results = pd.read_csv(additional_path + "train_with_cv_results.csv")[['location_id', 'flood_probability']]
test_with_cv_results = pd.read_csv(additional_path + "test_with_cv_results.csv")[['location_id', 'flood_probability',]]
submission = pd.read_csv(base_path + "SampleSubmission.csv")
images = np.load(base_path + "composite_images.npz")
display(train.head(), train.shape, train_with_cv_results.head(), train_with_cv_results.shape, test.head(), test.shape)

In [ ]:
train[train['label']== 1].describe()

In [ ]:
train[train['label']== 0].describe()

In [ ]:
def get_location(value):
  return value.split("_")[0] + '_' + value.split("_")[1]

def get_event_id(value):
  return value.split("_")[3]
for df in [train, test]:

  df['location_id'] = df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
  df['event_idx'] = df.groupby('location_id', sort=False).ngroup()

  df['event_t'] = df.groupby('location_id').cumcount()

print(len(set(train['location_id'])), len(set(test['location_id'])))
print(len(set(train['location_id']).intersection(set(test['location_id']))))
display(train.head(), test.head())

In [ ]:
train.groupby(['location_id'])['event_id'].count()

* each image has 730 events
* no intersection of images betweeen the two data sets (unique sets)
* The numpy files has 898 images for both train and test

* The images are annual cloud-free composite images from Sentinel-2 satellite imagery. They are of size 128x128 and contain the following 6 channels:

      Sentinel-2 B2 (Blue)
      Sentinel-2 B3 (Green)
      Sentinel-2 B4 (Red)
      Sentinel-2 B8 (NIR)
      Sentinel-2 B11 (SWIR)
      Slope (derived from NASA SRTM)

* the images are essentially static for any event/location pair over the study period.
  * the images only serve as spatial representations of the environment for that location over the 730 day period
  * it reflects static or semi-static environmental conditons (e.g land use, vegetation, water bodies, topography) that could influence flood occurence
  * so the images cannot provide temporal insights but what we can do is extract spatial features such as NDVI, NDWI, NDBI, Topographic features like slope and elevation changes from the slope channel
  * combine the spatial features with temporal precipitation data to enrich the dataset by treating the spatial features as fixed covariates that describe each location.
    * Areas with high NDWI Might flood more frequently with heavy precipitation
    * LOcations with high slope values might experience flash floods after intense rainfall

  * Image processing:
    * Use pretrained models to extract image embeddings or use PCA for dimensionality reduction
    * create a binary classifier where 1 is images where a flood has occured in any of the 730 events and 0 if no floods has occured to create a soft flag for flood-prone locations. Even if not perfect they can serve as a proxy for environmental vulnerability to floods
    * The image classifier naturally reduces the extreme imbalance in the dataset by focusing on binary flood/non-flood classification
  
  * clustering locations:
    * group events/locations based on spatial features (e.g NDVI, NDWI) to identify patterns in flood susceptibility
  * correlating spatial features with precipitation thresholds:
    * study how spatial features interact with specific precipitation thresholds that leads to floods

  * You can think of event_id_X_1 being the 01/01/2024 and event_id_X_2 being 02/01/2024 (dd/mm/yyyy).

### Data Preprocessing + Feature Engineering

In [ ]:
train_df = pd.merge(train, train_with_cv_results, on='location_id', how='left')
test_df = pd.merge(test, test_with_cv_results, on='location_id', how='left')


display(train_df.head(), train_df.shape, test_df.head(), test_df.shape)

In [ ]:
selected_columns = []
n_splits = 10
gkf = StratifiedGroupKFold(n_splits = n_splits)

train_df['fold'] = -1
for fold, (_, val_idx) in enumerate(gkf.split(train_df, train_df['label'], groups = train_df['location_id'])):
    train_df.loc[val_idx, "fold"] = fold

train_df['fold'].value_counts()

### More Feature Engineering

In [ ]:
def get_date_like_features(data):
  data['event_t'] = data['event_t'].astype(int)
  data['event_t_5_window'] = data['event_t'] // 3
  return data

In [ ]:
# train_df.groupby(['location_id'])['label'].agg('count')

In [ ]:
sns.displot((train['precipitation']))

In [ ]:
sns.displot(np.log1p(test['precipitation']))

In [ ]:
train['event_t'].describe()

In [ ]:
train[train['label']==1]['event_t'].describe()

In [ ]:
train['event_t'].describe()

In [ ]:
train[(train['precipitation']==0) & (train['label']==1)]['event_t'].describe()

In [ ]:
train[(train['precipitation']!=0) & (train['label']==1)]['event_t'].describe()

In [ ]:
train['location_id'].nunique()

In [ ]:
from types import new_class
def apply_expanding_combinations(df, group_cols_list, target_col='Sales', shift_periods=[1], min_periods=1, stats=['mean', 'std']):
    # Loop through the group column combinations
    for group_cols in group_cols_list:
        # Generate base name for the grouping
        group_name = '_'.join(group_cols)

        for shift_period in shift_periods:
            for stat in stats:
                expanding_col_name = f'expanding_grouped_{group_name}_{target_col}_shift_{shift_period}_{stat}'

                # Apply groupby, shift, and expanding for the given statistic
                df[expanding_col_name] = df.groupby(group_cols)[target_col].transform(
                    lambda x: x.shift(shift_period).expanding(min_periods=min_periods).agg(stat)
                )

    return df

def smoothen_target(df, group_cols, target_col):
  n_std = 10
  for i_smooth in [target_col]:
      df_id_outlier = df.groupby(group_cols,as_index=False).agg({
          f'{i_smooth}': lambda x: x.mean() + n_std*x.std()
      }).rename(columns={f'{i_smooth}':f'{i_smooth}_outlier'})

      df_id_mean = df.groupby(group_cols,as_index=False).agg({
          f'{i_smooth}': 'mean'
      }).rename(columns={f'{i_smooth}':f'{i_smooth}_mean'})

      df = df.merge(df_id_outlier, on=group_cols[0], how='left')
      df = df.merge(df_id_mean, on=group_cols[0], how='left')

      df[f'{i_smooth}'] = np.where(
          df[f'{i_smooth}'] > df[f'{i_smooth}_outlier'],
          df[f'{i_smooth}_mean'],
          df[f'{i_smooth}']
      )

  return df


def create_rolling_features(data, group_cols, target_col, windows, shift_period, min_period, statistics):
    def apply_statistic(x, stat):
        rolled = x.shift(shift_period).rolling(window=window, min_periods=min_period)
        if stat == 'mean':
            return rolled.mean()
        elif stat == 'median':
            return rolled.median()
        elif stat == 'std':
            return rolled.std()
        elif stat == 'min':
            return rolled.min()
        elif stat == 'max':
            return rolled.max()
        elif stat == 'skew':
            return rolled.skew()
        elif stat == 'sum':
            return rolled.sum()
        elif stat == 'quantile':
            return rolled.quantile(0.95)

        elif stat.startswith('quantile_'):
            q = float(stat.split('_')[1])
            return rolled.quantile(q)
        else:
            raise ValueError(f"Unknown statistic: {stat}")

    for window in windows:
        for stat in statistics:
            stat_name = stat if not stat.startswith('quantile_') else f"{stat.split('_')[1]}th"
            col_name = f'rolling_previous_grouped_{target_col}_{stat_name}_{window}_{shift_period}'

            data[col_name] = data.groupby(group_cols)[target_col].transform(
                lambda x: apply_statistic(x, stat)
            )

    return data



def custom_agg(x):
    return x.max() - x.min()

def get_date_features(df):
  # Simulate year (assuming 365 days per year)
  df['year'] = (df['event_t'] // 365) + 1  # Year 1 or 2

  # Simulate month (approximate)
  df['month'] = ((df['event_t'] % 365) // 30) + 1  # 30-day months approximation

  # Simulate week of the year
  df['week_of_year'] = (df['event_t'] % 365) // 7 + 1

  # Simulate day of the month
  df['day_of_month'] = (df['event_t'] % 30) + 1  # Assuming 30-day months

  # Simulate day of the week (0 = Monday, 6 = Sunday)
  df['day_of_week'] = df['event_t'] % 7

  # Simulate quarter
  df['quarter'] = ((df['month'] - 1) // 3) + 1
  return  df




def feature_engineering(train, test):
  data = pd.concat([train, test])
  data.sort_values(by = ['location_id', 'event_t'], inplace=True)
  data['event_t'] = data['event_t'].astype(int)
  # data = smoothen_target(data, ['location_id'], 'precipitation')

  data['event_binary'] = data['event_t'].apply(lambda x: 1 if (x >= 296 and x <= 435) else 0)


  group_cols =['location_id']
  # data = apply_expanding_combinations(
  #     data,
  #     [group_cols],
  #     target_col='precipitation',
  #     shift_periods=[1],#1,3, 4, 5, 6, 7, 8, 24
  #     min_periods=1,
  #     stats=['mean']
  # )

  statistics = ['mean'] #, 'median', 'std', 'quantile_0.25', 'quantile_0.75'
  min_period = 1

  shift_period = 0
  windows = [3, 4,10,20, 25, 30,55,60, 75, 296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 2
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 3
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 4
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 5
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 6
  # windows = [ 3,4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 8
  # windows = [ 3,4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)
  # # data = get_date_features(data)

  for col in ['precipitation']:
    # data[f"grouped_location_{col}_cum"] = data.groupby('location_id')[col].cumsum().shift(1)

    # quantile = 0.95  # Define the quantile you want to calculate
    # for stat in ['mean', 'quantile']:
    #     if stat != 'quantile':
    #         data[f"location_grouped_{col}_{stat}"] = data.groupby('location_id')[col].transform(stat)
    #         data[f"diff_{col}_{stat}"] = data[col] - data[f"location_grouped_{col}_{stat}"]


    for shift in range(1,365):
      data[f'{col}_shift_{shift}'] = data.groupby('location_id')[col].shift(shift)
      data[f'{col}_next_shift_{shift}'] = data.groupby('location_id')[col].shift(-shift)




    # for window in windows:
    #   data[f'{col}_rolling_grouped_custom_{window}'] = (
    #       data.groupby('location_id')[col]
    #       .rolling(window)
    #       .apply(custom_agg)
    #       .reset_index(level=0, drop=True)  # Reset the index to align with the original DataFrame
    #   )

    for span in [7]:
        data[f'{col}_ewm_grouped_mean_{span}'] = (
            data.groupby('location_id')[col]
            .ewm(span=span, adjust=False)
            .mean()
            .reset_index(level=0, drop=True)  # Reset the index to align it with the original DataFrame
        )




  train = data[data['label'].notna()].reset_index(drop = True)
  test = data[data['label'].isna()].reset_index(drop = True)

  return train, test

new_train, new_test = feature_engineering(train_df, test_df)
display(new_train.head(), new_train.shape, new_test.head(), new_test.shape)

In [ ]:
sns.distplot(new_test['precipitation'])

### MODELLING
674 224

In [ ]:
new_train['label'].value_counts()

In [ ]:
for i in range(n_splits):
  print(new_train[new_train['fold'] == i]['label'].value_counts())
  print("-"* 100)

### MODELLING

In [ ]:
new_test.isnull().sum()

In [ ]:

indices_cols = [
  'EVI_mean',
 'EVI_median',
 'EVI_std',
 'MNDWI_mean',
 'MNDWI_median',
 'MNDWI_std',
 'MSI_mean',
 'MSI_median',
 'MSI_std',
 'NDVI_mean',
 'NDVI_median',
 'NDVI_std',
 'NDWI_mean',
 'NDWI_median',
 'NDWI_std',
 'Slope_mean',
 'Slope_median',
 'Slope_std',
]

selected_columns =['precipitation','flood_probability','event_binary', 'event_t'] + [col for col in new_train if 'diff' in col or 'shift' in col or 'grouped' in col ]

print(selected_columns)
target_col = 'label'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from scipy.stats import hmean, gmean

test_preds_list = []  # Store fold-wise test predictions
loglosses = []
oof = []

new_train = new_train.fillna(0)
new_train[selected_columns] = new_train[selected_columns].astype(np.float32)

new_test = new_test.fillna(0)
new_test[selected_columns] = new_test[selected_columns].astype(np.float32)

# K-Fold Training
for fold in range(n_splits):
    print(f"============================= TRAINING FOLD: {fold+1} ================================")

    training = new_train[new_train['fold'] != fold]
    validation = new_train[new_train['fold'] == fold]


    y_train = training[target_col]
    y_test = validation[target_col]
    X_train = training[selected_columns]
    X_test = validation[selected_columns]


    # Train the TabNet model
    model = TabNetClassifier(n_d = 35, # Width of the decision prediction layer
                                n_a = 35, # Width of the attention embedding for each mask
                                n_steps = 2, # Number of steps in the architecture
                                gamma = 1.3, # coefficient for feature reusage in the masks
                                n_independent = 1, # Number of independent Gated Linear Units layers at each step
                                n_shared = 2, # Number of shared Gated Linear Units at each step
                                momentum = 0.02, # Momentum for batch normalization
                                clip_value = None, # extra sparsity loss coefficient
                                lambda_sparse = 1e-3,
                                optimizer_fn = torch.optim.AdamW, # Adam optimizer
                                optimizer_params = dict(lr = 2e-2, weight_decay=0.01), # Optimizer params
                                scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
                                scheduler_params = {'T_0':5,
                                                    'eta_min':1e-3,
                                                    'T_mult':1,
                                                    'last_epoch':-1},
                                mask_type = 'entmax', # masking function to use for selecting features
                                device_name='cuda', # Run on gpu
                                seed = 2025)

    model.fit(X_train.values, y_train.values.ravel(), eval_set = [(X_test.values, y_test.values.ravel())],
                  max_epochs = 40,
                  patience = 40,
                  virtual_batch_size = 256,
                  batch_size = 4096,
                  eval_metric=['logloss'])

    # Store out-of-fold predictions
    oof_preds = model.predict_proba(X_test.values)[:, 1]

    # Calculate logloss
    loss = log_loss(y_test, oof_preds)
    print(f"Logloss: {loss}")
    print("*" * 100)
    loglosses.append(loss)

    # Inference on the test set
    fold_preds = model.predict_proba(new_test[selected_columns].values)[:, 1]
    test_preds_list.append(fold_preds)  # Store for alternative means

    df = new_train.loc[validation.index.values, ['event_id', 'location_id', 'flood_probability', 'event_t', 'label']].copy()
    df['oof_tabnet_pred'] = oof_preds
    oof.append(df)


# Compute final scores
# new_train['oof_correct_prob'] = oof_preds
oof = pd.concat(oof, axis=0, ignore_index=True)
oof.to_csv(f'tabnet_oof_koleshjr_version{VERSION}.csv', index=False)
print(f"Mean logloss: {np.mean(loglosses)}")
print(f"OOF logloss: {log_loss(oof['label'], oof['oof_tabnet_pred'])}")
print(f"OOF roc_auc: {roc_auc_score(oof['label'], oof['oof_tabnet_pred'])}")

# Convert list of fold predictions to NumPy array (shape: n_splits x n_samples)
fold_preds_array = np.array(test_preds_list)

# Compute different mean types
avg_preds_arith = fold_preds_array.mean(axis=0)  # Arithmetic mean




# Save different mean-based submissions
submission = new_test[['event_id']].copy()
submission['tabnet_preds'] = avg_preds_arith
submission.to_csv(f'tabnet_version{VERSION}.csv', index=False)

### Normalizing the Probabilities based on the Flood Probability

In [ ]:
from sklearn.metrics import log_loss

print(f"logloss before normalizing: {log_loss(oof['label'], oof['oof_tabnet_pred'])}")

locations_to_normalize = oof[oof['flood_probability'] >= 0.5]['location_id'].unique()
oof['oof_sum_prob'] = oof.groupby('location_id')['oof_tabnet_pred'].transform('sum')

# Avoid division by zero
epsilon = 1e-8
oof['oof_tabnet_norm'] = oof['oof_tabnet_pred']  # Copy original values

oof.loc[oof['location_id'].isin(locations_to_normalize), 'oof_tabnet_norm'] = (
    oof.loc[oof['location_id'].isin(locations_to_normalize), 'oof_tabnet_pred'] /
    (oof.loc[oof['location_id'].isin(locations_to_normalize), 'oof_sum_prob'] + epsilon)
)

print(f"logloss after normalizing: {log_loss(oof['label'], oof['oof_tabnet_norm'])}")